# LightGBM + Optuna

## Optuna 
* 하이퍼파라미터 튜닝에 쓰고 있는 최신 AutoML기법
* 빠르게 튜닝이 가능하다는 장점이 있음
* 하이퍼파라미터 튜닝 방식을 지정할 수 있음 -> 직관적인 api인 튜닝된 lightgbm도 제공함
* 다른 라이브러리들에 비해 직관적인 장점이 잇어 코딩하기에 용이함 

![](img/2022-12-01-17-30-18.png)

In [2]:
import numpy as np
import pandas as pd
import optuna
from lightgbm import LGBMClassifier
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

## 전처리

In [3]:
path = 'C:/Users/JiwonMoon/Desktop/Data/open/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')

train = train.drop(["index"], axis=1)
train.fillna("NAN", inplace=True)
test = test.drop(["index"], axis=1)
test.fillna("NAN", inplace=True)

In [4]:
train_ohe = pd.get_dummies(train)
test_ohe = pd.get_dummies(test)

In [7]:
X = train_ohe.drop(["credit"], axis=1)
y = train["credit"]
X_test = test_ohe.copy()

In [10]:
def objective(trial: Trial) -> float:
    params_lgb = {
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "objective": "multiclass",
        "metric": "multi_logloss",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

    model = LGBMClassifier(**params_lgb)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    )

    lgb_pred = model.predict_proba(X_valid)
    log_score = log_loss(y_valid, lgb_pred)
    
    return log_score

In [11]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2022-12-01 17:34:56,169] A new study created in memory with name: lgbm_parameter_opt
c:\Users\JiwonMoon\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\JiwonMoon\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-12-01 17:34:59,262] Trial 0 finished with value: 0.7430471798030405 and parameters: {'reg_alpha': 1.12424581642324e-05, 'reg_lambda': 0.08556428806974939, 'max_depth': 15, 'num_leaves': 154, '

Best Score: 0.7301963981351943
Best trial: {'reg_alpha': 9.145366937509386e-06, 'reg_lambda': 0.008790499283853408, 'max_depth': 14, 'num_leaves': 114, 'colsample_bytree': 0.47322294090686734, 'subsample': 0.6466238370778892, 'subsample_freq': 1, 'min_child_samples': 92, 'max_bin': 277}


In [12]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [13]:
# 파라미터들관의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [14]:
# 각 파라미터들의 상관관계
optuna.visualization.plot_contour(
    study,
    params=[
        "max_depth",
        "num_leaves",
        "colsample_bytree",
        "subsample",
        "subsample_freq",
        "min_child_samples",
        "max_bin",
    ],
)

In [15]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

🔎 결과해석: <br/>
그동안 머신러닝 알고리즘을 사용해서 하이퍼파라미터 서치를 할때 GridSearchCV나 RadomizedSearchCV를 사용했었다. 그러나 GridSearchCV는 시간이 정말 많이 걸리고, RandomizedSearchCV는 그것보단 적게 걸리긴하지만 그래도 시간이 오래걸렸다. 그런데 이번에 Optuna라는 AutoML 기법을 사용했는데, 하이퍼파라미터들의 관계나 중요도를 파악할수 있어서 정말 좋았다. 무엇보다 수행시간을 보면 알수 있겠지만 정말 매우매우 빠른 속도로 파라미터를 서치해준다. 앞으로 잘 이용하게 될 패키지 인것 같아 튜토리얼에 포함했다. 